In [1]:
import os, json
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# File paths
input_folder_path = 'C:/SHB0004/Resources/'
final_output_file_path = 'C:/SHB0004/Consolidated_Entity List2.xlsx'
levels_json_file = 'C:/SHB0004/Levels_Revised.json'
files = os.listdir(input_folder_path)

In [3]:
# Function to check date format and return True if valid, False otherwise
def validate_and_convert_date(date_str):
    if date_str.strip():  # Check if not empty or null
        try:
            # Use '%m/%d/%y' format for validation and conversion
            return datetime.strptime(date_str, '%m/%d/%y')
        except ValueError:
            return None  # Return None for invalid date strings
    return None  # Return None for empty strings or non-string values

In [4]:
def join_unique(x):
    return ', '.join(set(filter(None, map(str.strip, x))))

def join_unique_address(x):
    return '/ '.join(set(filter(None, map(str.strip, x))))

In [5]:
dtype_dict = {
    'Associated Docs (People)': str,
    'People Tracker Control Number': str,
    'First Name or Initial': str,
    'Middle Name or Initial': str,
    'Last Name': str,
    'City': str,
    'State': str,
    'ZIP Code': str,
    'Date of Birth': str,
    'Financial Account Number': str
}

In [6]:
# List to store DataFrames
dfs = []

In [7]:
# Loop through each file and read it into a DataFrame, then append to the list
for file in files:
    if file.endswith('.xlsx') or file.endswith('.xlsx'):  # Check if the file is an Excel file
        file_path = os.path.join(input_folder_path, file)  # Get the full file path
        df = pd.read_excel(file_path, dtype=dtype_dict)
        dfs.append(df)


In [8]:
# Combine DataFrames
combined_df = pd.concat(dfs, ignore_index=True)

In [9]:
# Removing Row Duplicates
combined_df.drop_duplicates(inplace=True)

In [10]:
#Replace NaN values with blanks
combined_df.fillna('', inplace=True)


In [ ]:
combined_df

In [12]:
# # Filter rows based on valid date format in 'Date of Birth' column
# combined_df['Date of Birth'] = combined_df['Date of Birth'].apply(validate_and_convert_date)
# # Assuming df_sorted is your DataFrame
# combined_df['Date of Birth'] = pd.to_datetime(combined_df['Date of Birth'], errors='coerce')
# combined_df['Date of Birth'] = combined_df['Date of Birth'].fillna(pd.to_datetime('1900-01-01'))

In [13]:
# # If you want to convert the 'Date of Birth' back to string format with the desired format
# combined_df['Date of Birth'] = combined_df['Date of Birth'].dt.strftime('%m/%d/%Y')

In [14]:
combined_df.fillna({'Suffix': '', 'State': '', 'ZIP Code': '', 'Country (if not USA)': ''}, inplace=True)

In [ ]:
combined_df

In [16]:
# Validating the ZIP Codes, State, City fields
valid_zip_mask = combined_df['ZIP Code'].str.match(r'^\d{5}$') | (combined_df['ZIP Code'] == '')
valid_state_mask = combined_df['State'].str.match(r'^[A-Z]{2}$') | (combined_df['State'] == '')
valid_city_mask = combined_df['City'].str.match(r'^[a-zA-Z\s]+$') | (combined_df['City'] == '')
filtered_df = combined_df[valid_zip_mask & valid_state_mask & valid_city_mask]
filtered_df.loc[:, 'Address'] = filtered_df['Address'].str.strip()

In [ ]:
filtered_df.head()

In [18]:
# Load the JSON file
with open(levels_json_file, 'r') as file:
    levels_data = json.load(file)

In [19]:
# Remove quotations around 'join_unique' in aggregation functions
for level_data in levels_data.values():
    aggregation_functions = level_data['aggregation_functions']
    for col, func in aggregation_functions.items():
        if func == 'join_unique':
            aggregation_functions[col] = join_unique
        if func == 'join_unique_address':
            aggregation_functions[col] = join_unique_address

In [20]:
# Iterate over the levels and perform aggregation
for level_name, level_data in levels_data.items():
    columns = level_data['columns']
    aggregation_functions = level_data['aggregation_functions']
    df_grouped = filtered_df.groupby(columns).agg(aggregation_functions).reset_index()
    # Perform further operations with df_grouped as needed
    print(f"Grouping by {level_name} with columns - {columns} completed.")


Grouping by level1 with columns - ['Social Security Number', 'First Name or Initial', 'Last Name', 'Suffix', 'Date of Birth'] completed.


In [ ]:
df_grouped

In [22]:
# # Split the 'firstname', 'lastname', 'middle name', and 'dob' columns by commas and expand them into separate columns
# df_split = pd.concat([df_grouped[col].str.split(',', expand=True).add_prefix(f'{col}_') for col in ['First Name or Initial', 'Last Name', 'Middle Name or Initial', 'Date of Birth']], axis=1)

In [23]:
# # Concatenate the split columns with the original DataFrame
# df_combined = pd.concat([df_grouped, df_split], axis=1)

In [24]:
# Write the DataFrame to an Excel file
df_grouped.to_excel(final_output_file_path, index=False)